In [115]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [116]:
from dotenv import load_dotenv

_ = load_dotenv()

In [117]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [118]:
class AgentState(TypedDict):
    # The 'messages' field stores the conversation history as a list of messages.
    # Each message is typically a dictionary with keys like 'role' and 'content',
    # e.g., {"role": "user", "content": "What is the capital of France?"}
    #
    # The 'Annotated[..., operator.add]' syntax tells frameworks like LangGraph
    # that when merging multiple AgentState objects (e.g. from parallel branches),
    # this field should be combined using list concatenation (list1 + list2)
    # instead of being overwritten.
    messages: Annotated[list[AnyMessage], operator.add]

In [119]:
# Or you can have a more complex state

from typing import Union


class AgentState(TypedDict):
    input: str
    messages: Annotated[list[AnyMessage], operator.add]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

```ascii
# Entry point of the graph
          ▼
       ┌──────┐
       │ "llm"│   ←←←←←←←←←←←←←←←←←←←←←←←←←←←←
       └──────┘                             ↑
           │                                │
           │ Generate Thought + Action      │
           ▼                                │
      Check if tool_call exists             │
           │                                │
      ┌────▼────┐                           │
      │ Exists? │──────No──────→──┐         │
      └─────────┘                │         │
           │                    │         │
          Yes                   │         │
           ▼                    │         │
       ┌────────┐               │         │
       │"action"│               │         │
       └────────┘               │         │
           │                    │         │
           │ Execute tools      │         │
           ▼                    │         │
   Wrap results as ToolMessage │         │
           │                    │         │
           └──────────────→─────┘         │
              Add edge: action → llm──────┘


```


In [120]:
from typing import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, END
from langchain_core.messages import SystemMessage, ToolMessage


class Agent:

    def __init__(self, model, tools, system=""):
        # Store optional system prompt
        self.system = system

        # Create a LangGraph state graph with AgentState (which contains messages)
        graph = StateGraph(AgentState)

        # Add LLM node, which handles generating Thought/Action/Answer
        graph.add_node("llm", self.call_openai)

        # Add Action node, which executes tool calls
        graph.add_node("action", self.take_action)

        # Add conditional logic:
        # If the LLM response contains tool calls, go to action node.
        # Otherwise, finish the graph.
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )

        # Create a loop: after tool execution, go back to the LLM
        graph.add_edge("action", "llm")

        # Set the starting node to be the LLM
        graph.set_entry_point("llm")

        # Compile the graph to a langchain runnable
        self.graph = graph.compile()

        # print ASCII diagram
        # self.graph.get_graph().print_ascii()

        # or get mermaid diagram
        # print(self.graph.get_graph().draw_mermaid())

        # Store tools as a name → tool map for dispatch
        self.tools = {t.name: t for t in tools}

        # Bind tools to the LLM so it knows the available actions
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        # Check if the latest message from AIMessage includes tool calls
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        # Prepare message history
        messages = state["messages"]

        # Prepend system prompt if available
        # since it's not stored in state["messages"] by default.
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages

        # Send messages to the LLM and get a new response (which may include tool calls)
        message: str = self.model.invoke(messages)

        # Return the new assistant message in the required update format
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        # Extract tool calls from the latest AI message.
        #
        # For example, the latest message (AIMessage) may look like this:
        #
        #   Tool Name: None
        #   Tool Calls:
        #     [
        #         {
        #             'name': 'average_dog_weight',
        #             'args': {'name': 'Border Collie'},
        #             'id': 'call_403boG61cKmxfUA59KtLYApF',
        #             'type': 'tool_call'
        #         },
        #         {
        #             'name': 'average_dog_weight',
        #             'args': {'name': 'Scottish Terrier'},
        #             'id': 'call_0zJOoGb4xZI3dNNuIo15Vuou',
        #             'type': 'tool_call'
        #         }
        #     ]
        #   Content: (empty at this stage)
        #
        # This indicates the LLM has decided to invoke two tool calls,
        # which we now need to execute.
        tool_calls = state["messages"][-1].tool_calls
        results = []

        for t in tool_calls:
            print(f"Calling: {t}")

            # Check if the tool name is valid
            if not t["name"] in self.tools:
                print("\n ....bad tool name....")
                result = (
                    f"bad tool name ${t["name"]}, retry"  # Let the LLM handle the error
                )
            else:
                # Execute the corresponding tool with the provided arguments
                result = self.tools[t["name"]].invoke(t["args"])

            # Wrap the result in a ToolMessage so the LLM can observe it
            results.append(
                ToolMessage(
                    tool_call_id=t["id"],  # Echo the tool call ID for tracking
                    name=t["name"],
                    content=str(result),
                )
            )

        print("Back to the model!")

        # Return all tool observations in the expected format
        return {"messages": results}

In [121]:
from langchain_core.tools import tool


@tool
def calculate(what: str) -> str:
    """Evaluate a Python expression. WARNING: only use on trusted input."""
    return str(eval(what))


@tool
def average_dog_weight(name: str) -> str:
    """Return the average weight of a dog based on its breed."""
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"

In [122]:
# prompt = """You are a smart research assistant. Use the search engine to look up information. \
# You are allowed to make multiple calls (either together or in sequence). \
# Only look up information when you are sure of what you want. \
# If you need to look up some information before asking a follow up question, you are allowed to do that!
# """

prompt = """You are a helpful and intelligent assistant that uses a step-by-step process to solve problems. 
You have access to tools and should use the following format in your natural language response:

Thought: Describe your reasoning
Action: tool name
Action Input: JSON-formatted arguments
Observation: (after tool output)
Final Answer: Your final conclusion

You must always include these sections in your response content when using a tool.

Available tools:
- average_dog_weight
- calculate
"""


model = ChatOpenAI(model="gpt-4o-mini")

tools = [calculate, average_dog_weight]
my_bot = Agent(model, tools, system=prompt)

In [123]:
from langchain_core.messages import HumanMessage

question = """I have 2 dogs, a Border Collie and a Scottish Terrier. \
What is their combined weight"""

initial_state = {"messages": [HumanMessage(content=question)]}

# Run the graph — LangGraph will automatically manage tool calls and LLM turns
final_state = my_bot.graph.invoke(initial_state)

# Extract the final assistant message
# print(final_state["messages"][-1].content)
for i, msg in enumerate(final_state["messages"]):
    print(f"[{i}] {msg.type.upper()}")
    print(f"Tool Name: {getattr(msg, 'name', None)}")
    print(f"Tool Calls: {getattr(msg, 'tool_calls', None)}")

    content = msg.content
    if content:
        print("Content:")
        # Add simple highlighting for ReAct-style patterns
        for line in content.split("\n"):
            if line.startswith("Thought:"):
                print(f"  🧠 {line}")
            elif line.startswith("Action:"):
                print(f"  🛠️  {line}")
            elif line.startswith("Action Input:"):
                print(f"  📦 {line}")
            elif line.startswith("Observation:"):
                print(f"  👀 {line}")
            elif line.startswith("Final Answer:"):
                print(f"  ✅ {line}")
            else:
                print(f"     {line}")
    else:
        print("Content: None")

    print("-" * 50)


# from IPython.display import Image

# Image(my_bot.graph.get_graph().draw_png())

Calling: {'name': 'average_dog_weight', 'args': {'name': 'Border Collie'}, 'id': 'call_hc4TcshtvrpyTEkRSwYWsEMS', 'type': 'tool_call'}
Calling: {'name': 'average_dog_weight', 'args': {'name': 'Scottish Terrier'}, 'id': 'call_WxQdFAw8fQrzs8h8GNagiC9U', 'type': 'tool_call'}
Back to the model!
[0] HUMAN
Tool Name: None
Tool Calls: None
Content:
     I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight
--------------------------------------------------
[1] AI
Tool Name: None
Tool Calls: [{'name': 'average_dog_weight', 'args': {'name': 'Border Collie'}, 'id': 'call_hc4TcshtvrpyTEkRSwYWsEMS', 'type': 'tool_call'}, {'name': 'average_dog_weight', 'args': {'name': 'Scottish Terrier'}, 'id': 'call_WxQdFAw8fQrzs8h8GNagiC9U', 'type': 'tool_call'}]
Content: None
--------------------------------------------------
[2] TOOL
Tool Name: average_dog_weight
Tool Calls: None
Content:
     a Border Collies average weight is 37 lbs
---------------------------------------------

In [129]:
tavily_search_tool = TavilySearchResults(max_results=4)  # increased number of results
print(type(tavily_search_tool))
print(tavily_search_tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [130]:
prompt = """You are a helpful and intelligent assistant that uses a step-by-step process to solve problems. 
You have access to tools and should use the following format in your natural language response:

Thought: Describe your reasoning
Action: tool name
Action Input: JSON-formatted arguments
Observation: (after tool output)
Final Answer: Your final conclusion

You must always include these sections in your response content when using a tool.

Available tools:
- average_dog_weight
- calculate
"""

model = ChatOpenAI(model="gpt-4o-mini")

tools = [tavily_search_tool]
my_bot_01 = Agent(model, tools, system=prompt)

In [134]:
final_state_01 = my_bot_01.graph.invoke(
    {
        "messages": [
            HumanMessage(
                content="What is the capital city of Australia, and what is the weather there today?"
            )
        ]
    }
)

for i, msg in enumerate(final_state_01["messages"]):
    print(f"[{i}] {msg.type.upper()}")
    print(f"Tool Name: {getattr(msg, 'name', None)}")
    print(f"Tool Calls: {getattr(msg, 'tool_calls', None)}")

    content = msg.content
    if content:
        print("Content:")
        # Add simple highlighting for ReAct-style patterns
        for line in content.split("\n"):
            if line.startswith("Thought:"):
                print(f"  🧠 {line}")
            elif line.startswith("Action:"):
                print(f"  🛠️  {line}")
            elif line.startswith("Action Input:"):
                print(f"  📦 {line}")
            elif line.startswith("Observation:"):
                print(f"  👀 {line}")
            elif line.startswith("Final Answer:"):
                print(f"  ✅ {line}")
            else:
                print(f"     {line}")
    else:
        print("Content: None")

    print("-" * 50)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'capital city of Australia'}, 'id': 'call_whQIH8DmG0T7qBABacDul2Kc', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Canberra weather today'}, 'id': 'call_7NHXjVvGmi4cSisRWX42uZce', 'type': 'tool_call'}
Back to the model!
[0] HUMAN
Tool Name: None
Tool Calls: None
Content:
     What is the capital city of Australia, and what is the weather there today?
--------------------------------------------------
[1] AI
Tool Name: None
Tool Calls: [{'name': 'tavily_search_results_json', 'args': {'query': 'capital city of Australia'}, 'id': 'call_whQIH8DmG0T7qBABacDul2Kc', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': 'Canberra weather today'}, 'id': 'call_7NHXjVvGmi4cSisRWX42uZce', 'type': 'tool_call'}]
Content: None
--------------------------------------------------
[2] TOOL
Tool Name: tavily_search_results_json
Tool Calls: None
Content:
     [{'title': 'List